In [ ]:
!git clone https://github.com/tabtoyou/KoLLaVA.git

Cloning into 'KoLLaVA'...
remote: Enumerating objects: 232, done.
remote: Counting objects: 100% (232/232), done.
remote: Compressing objects: 100% (215/215), done.
remote: Total 232 (delta 46), reused 145 (delta 9), pack-reused 0
Receiving objects: 100% (232/232), 2.01 MiB | 2.96 MiB/s, done.
Resolving deltas: 100% (46/46), done.


In [ ]:
%cd KoLLaVA

/content/KoLLaVA


In [ ]:
!pip install --upgrade pip
!pip install -e .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/KoLLaVA
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/transformers.git (to revision cae78c46) to /tmp/pip-install-l1mh0wku/transformers_3e095c8888a742799fc6d527155f9f97
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-install-l1mh0wku/transformers_3e095c8888a742799fc6d527155f9f97
  Running command git checkout -q cae78c46
  Resolved https://github.com/huggingface/transformers.git to commit cae78c46
  Installing build dependencies ... done
  Getting requirements t

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os
from llava.conversation import conv_templates, SeparatorStyle
from llava.utils import disable_torch_init
from transformers import CLIPVisionModel, CLIPImageProcessor, StoppingCriteria
from llava.model import *
from llava.model.utils import KeywordsStoppingCriteria

from PIL import Image

import os
import requests
from PIL import Image
from io import BytesIO


DEFAULT_IMAGE_TOKEN = "<image>"
DEFAULT_IMAGE_PATCH_TOKEN = "<im_patch>"
DEFAULT_IM_START_TOKEN = "<im_start>"
DEFAULT_IM_END_TOKEN = "<im_end>"

In [ ]:
def load_image(image_file):
    if image_file.startswith('http') or image_file.startswith('https'):
        response = requests.get(image_file)
        image = Image.open(BytesIO(response.content)).convert('RGB')
    else:
        image = Image.open(image_file).convert('RGB')
    return image

In [ ]:
model_name = 'tabtoyou/KoLLaVA-KoVicuna-7b'

# Load Model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = LlavaLlamaForCausalLM.from_pretrained(model_name, low_cpu_mem_usage=True, torch_dtype=torch.float16, use_cache=True).cuda()

Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPVisionModel: ['text_model.encoder.layers.3.self_attn.k_proj.bias', 'text_model.encoder.layers.2.self_attn.q_proj.bias', 'text_model.encoder.layers.0.mlp.fc1.weight', 'text_model.encoder.layers.8.layer_norm2.bias', 'text_model.encoder.layers.8.self_attn.out_proj.bias', 'text_model.encoder.layers.4.self_attn.out_proj.weight', 'text_model.encoder.layers.7.self_attn.q_proj.weight', 'text_model.encoder.layers.7.self_attn.v_proj.weight', 'text_model.encoder.layers.3.layer_norm1.weight', 'text_model.encoder.layers.8.self_attn.k_proj.weight', 'text_model.encoder.layers.7.mlp.fc2.weight', 'text_model.encoder.layers.0.self_attn.q_proj.bias', 'text_model.encoder.layers.11.self_attn.k_proj.bias', 'text_model.encoder.layers.11.layer_norm2.weight', 'text_model.encoder.layers.4.self_attn.out_proj.bias', 'text_model.encoder.layers.10.layer_norm1.bias', 'text_model.encoder.layers.3.mlp.fc1.weight',

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
image_processor = CLIPImageProcessor.from_pretrained(model.config.mm_vision_tower, torch_dtype=torch.float16)

mm_use_im_start_end = getattr(model.config, "mm_use_im_start_end", False)
tokenizer.add_tokens([DEFAULT_IMAGE_PATCH_TOKEN], special_tokens=True)
tokenizer.add_tokens([DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN], special_tokens=True)

vision_tower = model.get_model().vision_tower[0]
vision_tower = CLIPVisionModel.from_pretrained(vision_tower.config._name_or_path, torch_dtype=torch.float16, low_cpu_mem_usage=True).cuda()
model.get_model().vision_tower[0] = vision_tower

vision_config = vision_tower.config
vision_config.im_patch_token = tokenizer.convert_tokens_to_ids([DEFAULT_IMAGE_PATCH_TOKEN])[0]
vision_config.use_im_start_end = mm_use_im_start_end
vision_config.im_start_token, vision_config.im_end_token = tokenizer.convert_tokens_to_ids([DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN])
image_token_len = (vision_config.image_size // vision_config.patch_size) ** 2

Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPVisionModel: ['text_model.encoder.layers.3.self_attn.k_proj.bias', 'text_model.encoder.layers.2.self_attn.q_proj.bias', 'text_model.encoder.layers.0.mlp.fc1.weight', 'text_model.encoder.layers.8.layer_norm2.bias', 'text_model.encoder.layers.8.self_attn.out_proj.bias', 'text_model.encoder.layers.4.self_attn.out_proj.weight', 'text_model.encoder.layers.7.self_attn.q_proj.weight', 'text_model.encoder.layers.7.self_attn.v_proj.weight', 'text_model.encoder.layers.3.layer_norm1.weight', 'text_model.encoder.layers.8.self_attn.k_proj.weight', 'text_model.encoder.layers.7.mlp.fc2.weight', 'text_model.encoder.layers.0.self_attn.q_proj.bias', 'text_model.encoder.layers.11.self_attn.k_proj.bias', 'text_model.encoder.layers.11.layer_norm2.weight', 'text_model.encoder.layers.4.self_attn.out_proj.bias', 'text_model.encoder.layers.10.layer_norm1.bias', 'text_model.encoder.layers.3.mlp.fc1.weight',

In [ ]:
def inference(qs, image_file) :
  image = load_image(image_file)

  qs = qs + '\n' + DEFAULT_IM_START_TOKEN + DEFAULT_IMAGE_PATCH_TOKEN * image_token_len + DEFAULT_IM_END_TOKEN

  conv_mode = "multimodal"
  conv = conv_templates[conv_mode].copy()
  conv.append_message(conv.roles[0], qs)
  conv.append_message(conv.roles[1], None)
  prompt = conv.get_prompt()
  inputs = tokenizer([prompt])

  image_tensor = image_processor.preprocess(image, return_tensors='pt')['pixel_values'][0]
  input_ids = torch.as_tensor(inputs.input_ids).cuda()

  stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2
  keywords = [stop_str]
  stopping_criteria = KeywordsStoppingCriteria(keywords, tokenizer, input_ids)

  with torch.inference_mode():
      output_ids = model.generate(
          input_ids,
          images=image_tensor.unsqueeze(0).half().cuda(),
          do_sample=True,
          temperature=0.2,
          max_new_tokens=1024,
          stopping_criteria=[stopping_criteria])

  input_token_len = input_ids.shape[1]
  n_diff_input_output = (input_ids != output_ids[:, :input_token_len]).sum().item()

  if n_diff_input_output > 0:
      print(f'[Warning] {n_diff_input_output} output_ids are not the same as the input_ids')
  outputs = tokenizer.batch_decode(output_ids[:, input_token_len:], skip_special_tokens=True)[0]
  outputs = outputs.strip()
  if outputs.endswith(stop_str):
      outputs = outputs[:-len(stop_str)]
  outputs = outputs.strip()
  print(outputs)

In [ ]:
# image_file = "https://llava.hliu.cc/file=/nobackup/haotian/code/LLaVA/llava/serve/examples/extreme_ironing.jpg"
image_file = "/content/냉장고.jpeg"
query = "이미지 속 재료들로 어떤 음식을 만들 수 있을까요?"

inference(query, image_file)

True
이미지에는 다양한 식품이 있는 냉장고가 있습니다. 냉장고에는 과일, 빵, 우유, 물, 주스, 달걀, 치즈, 버터, 베리류, 블루베리, 딸기, 당근 등 다양한 식품이 들어 있습니다. 이러한 재료를 사용하여 다양한 요리를 만들 수 있습니다. 몇 가지 가능한 옵션은 다음과 같습니다:

1. 과일 샐러드: 베리류, 딸기, 당근을 섞어 신선하고 건강한 샐러드를 만들 수 있습니다.
2. 과일 베이스 스무디: 베리류, 딸기, 당근을 섞어 상큼한 스무디를 만들어 보세요.
3. 과일 샌드위치: 베리류, 딸기, 당근을 샌드위치 빵에 얹어 맛있는 샌드위치를 만들어 보세요.
4. 과일 디저트: 베리류, 딸기, 당근을 섞어 과일 코블러나 팬케이크와 같은 달콤한 디저트를 만들어 보세요.
5. 과일 기반 요리: 베리류, 딸기, 당근을 샐러드나 수프에 넣어 맛있는 요리를 만들어 보세요.

이는 몇 가지 예에 불과하며, 냉장고에 있는 다양한 식품을 사용하여 다양한 요리를 만들 수 있는 가능성은 무궁무진합니다.
